# Statistical tests with R

##### Guilherme Gimenez Jr

## Agenda

1. **About Hypothesis Testing**
2. **Importing our data**
    1. **Setting up hypothesis**
    2. **Jumping right into it**

### 1. About Hypothesis Testing

Building a hypothesis is one of the first steps in starting in the analysis of experiments, it is all about **answering a question**. Of course, before building a hypothesis is always nice to conduct an Exploratory Data Analysis (EDA) to get more insight into what information is available and what lies within the data.

#### What is a hypothesis?

Well, a hypothesis can be thought of as **an educated guess** about something in the data, it **must be testable** either by an experiment or observation**.

When we are proposing a hypothesis we should **write a statement**:

> [...] **if** and **then** [...] are necessary in formalized hypothesis. [...] In a formalized hypothesis, a tentative relationship is stated. [...] Formalized hypotheses contain two variables. One is "independent" and the other is "dependent". The independent variable is the one you, the "scientist" control, and the dependent variable is the one that you observe and/or measure the results. [University of California](https://www.csub.edu/~ddodenhoff/Bio100/Bio100sp04/formattingahypothesis.htm)

A very nice example of a well-written hypothesis:

- **If** skin cancer is **related** to ultraviolet light, **then** people with high exposure to UV light will have a higher frequency of skin cancer.

Notice that we have included the **dependent** variable, _skin cancer_ , the **independent variable**, _UV light_ , and also the expectations, or results, for an experiment, _higher frequency of skin cancer_.

#### What is Hypothesis Testing?

In simple words, we are **test the odds of our results happening by chance** (or if we have meaningful results). In order to do this we need to have a **null hypothesis $H_{0}$** (and an alternative hypothesis $H_{1}$) that we will try to reject or accept.

This is where the most known _p-value_ term is born: in order to determin the statistical significance of the results we analyse the _p-value_ , if is less than or equal to a _particular threshold_ there is evidence against the null hypothesis. Different fields use different threshold values when performing hypothesis testing, in our case we will use $\alpha=0.05$.

> Under the null hypothesis, a parameter of interest is set to a particular value, typically zero, which represents the "no effect" relative to the effectt the research is testing for. [Too Big to Fail: Large Samples and the p-value Problem](https://www.researchgate.net/publication/270504262_Too_Big_to_Fail_Large_Samples_and_the_p-Value_Problem)

The following table shows the possible outcomes:

|||Actual Validity of $H_{0}$|Actual Validity of $H_{0}$|
|-|-|-|-|
|||**$H_{0}$ is true**|**$H_{0}$ is false**|
|**Decision Made**|**Accept $H_{0}$**|True Negative|False Negative (Type II error)|
|**Decision Made**|**Reject $H_{0}$**|False Positive (Type I error)|True Positive|

The _p-value_ is a _very slippery terrain_ and must be dealt with caution. There are some problems related hypothesis testing like:

- A high number of observations can lead to significant _p-values_ even if there is no statistical significance
- Selective reporting and _p-hacking_ are some issues that arrive due to heavy usage of p-values

#### Types of statistical tests 

The appropriate statistical test for the data depend on **the number and type of variables** that will be included in the analysis.

There are [several tables](https://stats.idre.ucla.edu/other/mult-pkg/whatstat/) that can help when choosing the right statistical test to perform. Here we are going to see statistical tests when dealing with **independent groups**.

|Nature of Dependent Variables|Test|
|-|-|

### 2. Importing our data

In order to perform our statistical tests we will use some datasets that are already availabe on R (with our vanilla instalation). Following the previous lecture, we are going to use the **mtcars dataset**.

In [ ]:
# Let's see which datasets are already available from our vanilla R installation
data()

#### About the Data

The **mtcars dataset** was extracted from the Motor Trend US Magazine, comprising fuel consumption and 10 aspects of automobile design and performance for 32 vehicles (1973-1974 models).

|feature name|description|
|-|-|
|mpg|Consumption in miles/gallon|
|cyl|Number of Cylinders|
|disp|Displacement|
|hp|Gross Horsepower|
|drat|Rear axle ratio|
|wt|Weight (1000 lbs)|
|qsec|1/4 mile time|
|vs|Engine (0=V-Shaped, 1=straight)|
|am|Transmission (0=automatic, 1=manual)|
|gear|Number of forward gears|
|carb|Number of carburetors|

In [ ]:
# Attaching the database let's us access each one of the variables
# without using the '$' atomic operator
attach(mtcars)

head(mtcars)
# If this doesn't work on RStudio, try running this command:
# data(mtcars)

In [ ]:
?mtcars

Let's explore our data a little bit, we need to understand our variables types and their distribution...

In [ ]:
# Getting statistical information about each variable
summary(mtcars)

Now that we know a little bit about our data and what each variable means we can create a whole bunch of hypotheses.

Some that we are going to see:

1. Weight plays a huge part on fuel consumption (miles/gallon)
2. No difference between transmissions and fuel consumption (miles/gallon) 
3. No difference between transmissions and horsepower
4. Dependence of the number of carburators and cylinders

#### Weight plays a huge part on fuel consumption

Thinking back to our physics classes, do you remember this equation?

$F=m*a$

It's the famous 2nd Newton's Law, you probably had some exercises where you used it to solve for the friction between the tire and the road, or concerning the static and the dynamic friction and how they impact the velocity of a car or it's acceleration. Here we can build the intuition for our hypothesis:

> **If** weight is related to fuel consumption, **then** as the weight increases, so does the fuel consumption.

Well, we can see here that we want to test the strength and direction of the linear relationship between *weight* and *fuel consumption*, but we also **need to be sure that relationship in the sample data is strong enough to use to model the relationship in the population**.

##### Tests for Correlation

- [Pearson Correlation](https://www.statisticshowto.com/probability-and-statistics/correlation-coefficient-formula/)
- [Spearman Correlation](https://statistics.laerd.com/statistical-guides/spearmans-rank-order-correlation-statistical-guide.php)
- [Kendall Correlation](https://www.statisticshowto.com/kendalls-tau)

Note that there are differences between rank-based approaches and the 'conventional' Pearson Correlation tests, especially concerning **monotonic and linear correations**.

**We are going to use the Pearson Correlation Test on this example!**

Because of that, **before testing for the linear correlation, we need to perform some preliminary analysis**:

1. The covariation is **linear**?
2. Data from each feature **is normal**?

In [ ]:
# To check for the covariation we will use a very simple approach of a scatter plot
x <- wt
y <- mpg

# We are using **raw R**, you will find many packages that perform
# all of these approaches and also a better viz in next lectures!
plot(
    x, y, 
    main = "Covariation for Weight and Miles Consumption",
    xlab = "Weight",
    ylab = "Miles/gallon",
    pch = 19, frame = FALSE
)
abline(lm(y ~ x, data = mtcars), col = "blue")

Awesome, it seems that our covariance is somewhat linear, so we can proceed!

Now, we will check if both variables have a normal distribution, this is already an hypothesis test (the first one we will see here) using the [**Shapiro-Wilk Normality Test**](https://www.statisticshowto.com/shapiro-wilk-test/) and also the [**Normal Q-Q Plot**](https://data.library.virginia.edu/understanding-q-q-plots/) which draws the correlation between a given sample and the normal distribution.

In [ ]:
# First, let's test for the weight (x)
shapiro.test(x)

In [ ]:
shapiro.test(y)

In [ ]:
par(mfrow=c(1,2))
qqnorm(x); qqline(x)
qqnorm(y); qqline(y)

Well, assuming our $\alpha=0.05$ we can assume normality from the Shapiro-Wilk test and also from the normal q-q plots shown above, so we can finally proceed to our hypothesis tests for the correlation between Miles/Gallon and Weight.

One of the nice things about the correlation test in R is that choosing among many different tests for correlation is as easy as providing a new string to the `method` parameter. We will choose two methods here: `pearson` and `kendall` rank correlation tests.

Also, here are the corresponding null and alternative hypotheses:

**Null Hypothesis $H_{0}$**: $\rho=0$

**Alternative Hypothesis $H_{1}$**: $\rho\neq0$

In [ ]:
cor.test(x, y, method="pearson")

Awesome! It seems that we have a **significant negative correlation** between weight and miles/gallon, meaning that as the weight increases, the miles/gallon decreases; in other words, the fuel consumpion increases with the weight of the car and we can travel less miles per gallon.

#### Hands-On

It looks like we can probably work on our first hands-on assignment. Take a look at what the displacement variable actually means:

![Engine Displacement](https://www.buzzle.com/images/diagrams/engine-displacement-measurement.jpg)

Can we create a new hypothesis using this information? Actually, we can create **a lot of hypothesis**, but let's build the following one:

##### Our hypothesis

> **If** displacement is related to horsepower, **then** as the displacement increases, so does the horsepower of the engine.

*Tip: if a variable fails for the normality test or the linear covariation we can still test for correlation using the **rank-based** approach.*

#### No difference between transmissions and fuel consumption (miles/gallon) 

This hypothesis comes from the common sense that automatic are less efficient than manual transmission engines.

> **If** transmission has no effect on fuel consumption, **then** there should be no difference in fuel consumption for different transmissions.

##### Tests for Difference

We will test for difference in means for both populations using a **t-test**, but we need to ask ourselves a question:

*"The variance between the two populations are supposed to be equal or different?"*

Answering this question will help us in choosing the right test:

1. [**Student's t-test**](https://www.datanovia.com/en/lessons/types-of-t-test/unpaired-t-test/students-t-test/)
2. [**Wench Two Sample t-test**](https://www.datanovia.com/en/lessons/types-of-t-test/unpaired-t-test/welch-t-test/)

Think of Wench's t-test being less restrictive than the Student's t-test because variances could be different.

Again, we need to ensure some conditions before actually performing the test:

- Ensure populations may have been drawed from a normal distribution

In [ ]:
# Here we are only making the values more human-readable
mtcars$fam = factor(mtcars$am, levels=c(0,1), labels=c("automatic","manual"))

In [ ]:
summary(mtcars$fam)

In [ ]:
attach(mtcars)  # Need to reattach because of new variable

In [ ]:
shapiro.test(mpg[fam == "manual"])

In [ ]:
shapiro.test(mpg[fam == "automatic"])

In [ ]:
par(mfrow=c(1,2))
qqnorm(mpg[fam == "manual"]); qqline(mpg[fam == "manual"])
qqnorm(mpg[fam == "automatic"]); qqline(mpg[fam == "automatic"])

In [ ]:
boxplot(mpg~fam, ylab="Miles/Gallon", xlab="Transmission")

Looks like we already have some evidence that the means are not equal, but our test will helps us to ensure that this difference is significant.

In [ ]:
t.test(mpg ~ fam)

Comparing the p-value with our $\alpha$ we see that the result is significant, meaning that there is a significant difference for both means. Remember that this alternative could be one-sided (greater or lesser).

Let's see if the more restrictive Student's t-test also provides us the same result:

In [ ]:
t.test(mpg ~ fam, var.eq = T)

Nice, we have the same results! We can also test for a different alternative:

1. upper: $\mu_{1}\gt\mu_{2}$
2. lower: $\mu_{1}\lt\mu_{2}$
3. two-tailed: $\mu_{1}\ne\mu_{2}$

In [ ]:
t.test(mpg~fam, var.eq = T, alternative = "less")

We can see that using the alternative hypothesis as the difference in means less than 0 results in that the miles/gallon for automatic transmission is less than manual transmission. Awesome!

#### No difference between transmissions and horsepower

Another thing that transmission may impact is about the horsepower of the car.

> **If** transmission has no effect on horspower, **then** there should be no difference in horspower for different transmissions.

In [ ]:
shapiro.test(hp[fam == "manual"])

In [ ]:
shapiro.test(hp[fam == "automatic"])

One of the conditions is not respected regarding normal distribution: the horsepower for manual transmission. Because of that we will need to use another test: a non-parametric tests and compare medians instead of means!

This is the [**Wilcoxon Rank Sum Test**](https://data.library.virginia.edu/the-wilcoxon-rank-sum-test/)!

In [ ]:
boxplot(hp ~ fam)

In [ ]:
wilcox.test(hp ~ fam, mu=0, alt="two.sided", conf.int=T, conf.level=0.95, paired=F, exact=F)

Comparing the p-value with our $\alpha$ we can see that the results are significant and, therefore, the medians are not the same! Again, we used the two-sided alternative, so they could be greater or lesser.

#### Hands-On

Let's look at our second hands-on assignement: can you check if transmission has effect on the qsec (time it took to travel 1/4 miles).

##### Our hypothesis

> **If** transmission has no effect on fuel consumption, **then** there should be no difference in fuel consumption for different transmissions.

#### Dependence of the number of carburators and cylinders

On a engine the carburator is responsible for mixing air and fueld for internal combustion, the mixture is then 'given' to a cylinder or a group of cylinders. Let's check if they are actually dependent on one another:

> **If** the number of carburators and the number of cylinders are dependent, **then** for a fixed number of carburators we will have a fixed number of cylinders.

##### Tests for Dependence

Here we are dealing with two categorical/ordinal variables, in order to test for dependence we will need to use:

- [**Chi-Square Test**](https://www.statisticshowto.com/probability-and-statistics/chi-square/)

In order to use this test we need to use a contingency table, that consists on the count of each category for both variables

In [ ]:
# Creating the contigency table 
contingency <- table(carb, cyl)

In [ ]:
contingency

In [ ]:
chisq.test(contingency)

This warning is related to small samples in our population, so we are having poor estimates; but fear not, another statistical test that we can use in this scenario is the [**Fisher Test**](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5426219/#__sec5title):

> Fisher's exact test is practically applied only in analysis of small samples but actually it is valid for all sample sizes. While the chi-squared test relies on an approximation, Fisher's exact test is one of exact tests. Especially when more than 20% of cells have expected frequencies < 5, we need to use Fisher's exact test because applying approximation method is inadequate. Fisher's exact test assesses the null hypothesis of independence applying hypergeometric distribution of the numbers in the cells of the table.

**(KIM, H. Statistical notes for clinical researchers: Chi-squared test and Fisher's exact test.)**

In [ ]:
fisher.test(contingency)

Awesome! We can see that we have a significant value, meaning that we successfully rejected the null hypothesis, meaning that both variables are dependent, or related to each other.

### Summing Up

We've seen a little bit for hypotesis testing with categorical variables:

|function|proportion|input|comments|
|-|-|-|-|
||||
||||
||||
||||
||||
|`chisq.test`|$\ge2$ proportions|matrix or contigency table|frequencies should have values greater than 5 for accuracy|
|`fisher.test`|$\ge2$ proportions|matrix or contigency table|may have some problems with large datasets or small values|